### 檢查是否有最新一集的漫畫

In [1]:
import requests
res = requests.get('https://www.manhuabei.com/manhua/haizeiwang/')
res.encoding = 'utf-8'
#res.text

In [2]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(res.text, 'lxml')
comics = []
for i in soup.select('ul#chapter-list-1 > li > a'):
    i = i.get('title').rstrip('话')
    i = int(re.sub("\D", " ", i))
    #print(i[:4])
    comics.append(i)

ValueError: invalid literal for int() with base 10: '763 764'

In [40]:
max(comics)

'990'

In [42]:
import pandas
comic_df = pandas.DataFrame(comics)
comic_df.columns = ['series']


In [43]:
import sqlite3 as lite
with lite.connect('comics.sqlite') as db:
    comic_df.to_sql('comics', con = db, if_exists='replace')

In [45]:

with lite.connect('comics.sqlite') as db:
    df2 = pandas.read_sql_query('select max(series) as max_series from comics', con = db)
df2['max_series'][0]



'990'

In [53]:
def check_comics(s):
    # 檢查資料庫最新的漫畫集數
    with lite.connect('comics.sqlite') as db:
        df2 = pandas.read_sql_query('select max(series) as max_series from comics', con = db)
        max_series = int(df2['max_series'][0])
    
    # 利用網路爬蟲檢查網路上最新的集數
    res = requests.get('https://www.manhuabei.com/manhua/haizeiwang/')
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'lxml')
    comics = []
    for i in soup.select('ul#chapter-list-1 > li > a'):
        i = i.get('title').rstrip('话')
        i = int(re.sub("\D", " ", i)[:4])
    #print(i[:4])
    comics.append(i)
    # 將還未抓取的集數放到 left_comics 之中
    left_comics = []
    for i in range(max_series, max(comics)):
        left_comics.append(i)
    # 將抓取到的集數，寫到資料庫中
    with lite.connect('comics.sqlite') as db:
        comic_df.to_sql('comics', con = db, if_exists='replace')
    return left_comics

check_comics()

[]

### 完整抓取漫畫程式
    

In [ ]:
left_comics = check_comics()
left_comics = [959]
for i in left_comics:
    getComic(i)

In [41]:
import requests
from bs4 import BeautifulSoup
import re
res = requests.get('https://www.manhuabei.com/manhua/haizeiwang/')
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'lxml')
comics = []
def get_new_chater():
    for i in soup.select('ul#chapter-list-1 > li > a'):
        i = i.get('href')
        comics.append(i)
    return 'https://www.manhuabei.com'+ comics[-1]
get_new_chater()

'https://www.manhuabei.com/manhua/haizeiwang/573021.html'

### 算總頁數

In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver=webdriver.Chrome('.\\chromedriver.exe')
def total_page():
    url = get_new_chater()
    driver.get(url)
    assert '海贼王' in driver.title
    links = driver.find_element_by_css_selector("#images")
    print(links.text)
    a = links.text
    total_page = a.strip(')').strip('(').strip('1').strip('/')
    driver.close()
    return total_page

(1/17)


In [56]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import selenium
driver=webdriver.Chrome('.\\chromedriver.exe')
url = get_new_chater()
driver.get(url)
assert '海贼王' in driver.title
try:
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, 'images')))
    links = driver.execute_script("document.querySelector('.comic_wraCon.autoHeight > #images > img').style.display='inline';")
    #element = selenium.webdriver.support.ui.WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, 'some_id')))
    #selenium.webdriver.support.ui.Select(element).select_by_value('1')
except Exception as ex:
    print(ex)
#links.text
print(links)
driver.close()



None


In [52]:
import time, os
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

comics = []
def get_new_chater():
    for i in soup.select('ul#chapter-list-1 > li > a'):
        i = i.get('href')
        comics.append(i)
    return 'https://www.manhuabei.com'+ comics[-1]

def getComic(s):
    driver = webdriver.Chrome('C:\\Users\\UX433\\Desktop\\爬蟲\\chromedriver.exe')
    if not os.path.exists(str(s)):
        os.mkdir(str(s))
    #pageurl = 'https://www.8899.click/online/comic-103.html?ch={}-{}'
    #https://www.manhuabei.com/manhua/haizeiwang/573021.html?p=3
    pagenum = total_page()
    pageurl = get_new_chater()+'?p={}'.format(1)
    driver.get(pageurl.format(s,1))
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    for i in range(pagenum):
        driver.get(pageurl.format(i))
        soup = BeautifulSoup(driver.get(pageurl), 'lxml')
        imgurl = 'https:' + soup.select_one('#TheImg').get('src')
        res = requests.get(imgurl)
        with open('{}/{}.jpg'.format(s, i+ 1), 'wb') as f:
            f.write(res.content)
        time.sleep(1)
    driver.close()

### Flask API

### 取得隱碼Code

In [3]:

client_id = 'lvAbi0iSWH0QD9JKzvfD54'
callback_url = 'http://127.0.0.1:5000/callback'
URL = f'https://notify-bot.line.me/oauth/authorize?response_type=code&client_id={client_id}&redirect_uri={callback_url}&scope=notify&state=NO_STATE'
print(URL)

https://notify-bot.line.me/oauth/authorize?response_type=code&client_id=lvAbi0iSWH0QD9JKzvfD54&redirect_uri=http://127.0.0.1:5000/callback&scope=notify&state=NO_STATE


### 取得授權Token

In [12]:
import requests

code = 'd4SkbfllqIIeVnRWyilW1F'
client_secret = 'T5C5KdKhZlExghhUh1B23yQ0pyrG9zNWG5noDVKZDv0'
headers = {
    'Content-type': 'application/x-www-form-urlencoded'
}

payload = {
 'code':code,
 'client_id': client_id, 
 'client_secret': client_secret,
 'redirect_uri': callback_url,
 'grant_type' : 'authorization_code'   
}

res = requests.post('https://notify-bot.line.me/oauth/token', data = payload, headers = headers)

In [13]:
res.text

'{"status":200,"message":"access_token is issued","access_token":"WZABnxxPYKzlUFwp3bAdCgpxmsalQZ94MncTrS6j8jQ"}'

### 自動發訊息

In [15]:
token = 'WZABnxxPYKzlUFwp3bAdCgpxmsalQZ94MncTrS6j8jQ'
headers = {
    'Content-type': 'application/x-www-form-urlencoded',
    'Authorization': f'Bearer {token}'
}

payload = {
 'message':'Has new onePiece chapter', 
}

res = requests.post('https://notify-api.line.me/api/notify', data = payload, headers = headers)